# 2020 FRC Zebra Path Data

In [10]:
import json

import ipywidgets as widgets
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
class ZPath():
    def __init__(self, path_data):
        self.event = path_data['event']
        self.match = path_data['match']
        self.blue = [team['team_key'] for team in path_data['zebra']['alliances']['blue']]
        self.red = [team['team_key'] for team in path_data['zebra']['alliances']['red']]
        paths = []
        for alliance in ['blue', 'red']:
            for team in path_data['zebra']['alliances'][alliance]:
                for axis in ['xs', 'ys']:
                    paths.append(team[axis])
        self.paths = np.array(paths)
        self.times = np.array(path_data['zebra']['times'])
        self.score = path_data['score']

class RoboPaths():
    def __init__(self, file):
        self.event_summary = None
        self.paths = None
        
        self._read_file(file)
        
    def __getitem__(self, idx):
        return self.paths[idx]
    
    def __len__(self):
        return len(self.paths)
    
    def _read_file(self, file):
        # Convert all file data to JSON -- assumes file is not too big to load into memory
        with open(file) as jlfile:
            self.paths = [json.loads(line) for line in jlfile]
        
        # Create Event Summary Dataframe -- shows all events checked for zebra data
        events = [path['event'] for path in self.paths]
        matches = [path['match'] for path in self.paths]
        zebra = [ 0 if path['zebra'] is None else 1 for path in self.paths]
        self.event_summary = (
            pd.DataFrame({'event': events, 'match': matches, 'path': zebra})
            .groupby('event')
            .agg(path_matches=('path', 'sum'))
        )
        
        # Remove matches with no Zebra path data
        self.paths = [ZPath(path) for path in self.paths if path['zebra'] is not None]


In [3]:
rp = RoboPaths('frc2020B.jsonl')

In [4]:
rp[0].times

array([0.000e+00, 1.000e-01, 2.000e-01, ..., 1.498e+02, 1.499e+02,
       1.500e+02])

In [5]:
idx = 534
plt.plot(rp[idx].paths[0], rp[idx].paths[1])
plt.grid()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:


def plot_field():
    plt.clf()
    fig, ax = plt.subplots()
    ax.set_axis_off()
    # Field Perimieter
    ax.add_patch(matplotlib.patches.Rectangle((0, 0), 52.438, 26.938, fill=False))
    # Initiation Lines
    ax.add_artist(matplotlib.lines.Line2D([10, 10], [0, 26.938], color='silver'))
    ax.add_artist(matplotlib.lines.Line2D([42.438, 42.438], [0, 26.938], color='silver'))
    
    # Target Zones
    ax.add_artist(matplotlib.lines.Line2D([0, 2.5], [21.049, 19.049], color='cornflowerblue'))
    ax.add_artist(matplotlib.lines.Line2D([2.5, 0], [19.049, 17.049], color='cornflowerblue'))
    ax.add_artist(matplotlib.lines.Line2D([52.438, 49.938], [9.888, 7.888], color='tomato'))
    ax.add_artist(matplotlib.lines.Line2D([49.938, 52.438], [7.888, 5.888], color='tomato'))
    
    # Loading Zones
    ax.add_artist(matplotlib.lines.Line2D([0, 2.5], [10.388, 7.888], color='tomato'))
    ax.add_artist(matplotlib.lines.Line2D([2.5, 0], [7.888, 5.388], color='tomato'))
    ax.add_artist(matplotlib.lines.Line2D([52.438, 49.938], [21.549, 19.049], color='cornflowerblue'))
    ax.add_artist(matplotlib.lines.Line2D([49.938, 52.438], [19.049, 16.549], color='cornflowerblue'))
    
    ax.plot(rp[idx].paths[0][:150], rp[idx].paths[1][:150])
    
    ax.set_aspect('equal')
    ax.set_xlim(-1, 55)
    ax.set_ylim(-1, 27)
    ax.set_title('Field')

In [12]:
out = widgets.Output()
with out:
    plot_field()
    plt.show()